<a href="https://colab.research.google.com/github/kjmobile/lb/blob/main/0_assignment_yourname.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment: Multiple Regression for Flight Delay Prediction (Final Version)
## Objective
The goal is to build a multiple regression model to predict flight arrival delays using two independent variables: departure delay and weather. We will learn how to use the formula feature in `statsmodels` to explicitly set a baseline for a categorical variable ('Weather').
#
### Steps:
1.  **Connect to Database & Engineer Variables:** Write an SQL query to calculate `departure_delay` and retrieve `weather_condition` data.
2.  **Data Cleaning:** Remove null values and extreme outliers to improve model stability.
3.  **Build the Multiple Regression Model:** Create the model using `departure_delay` and `weather_condition` as independent variables, setting 'Clear' as the baseline for weather.
4.  **Analyze the Results:** Check the model's statistical significance and the impact of each variable on arrival delay.
5.  **Visualize the Results:** Use `seaborn` to visually analyze the model's performance and the effect of weather.


In [ ]:
!pip install pymysql

In [ ]:
# Step 0: Import necessary libraries
import pymysql
import pandas as pd
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')  # Hide warning messages for cleaner output


## Step 1: Connect to DB and Load Data
We connect to the database and load data from the flights_delay table which contains all the engineered features we need.


In [ ]:
# Create a direct connection to the database
conn = pymysql.connect(
    host='database-klee.cbgcswckszgl.us-east-1.rds.amazonaws.com',
    user='erau',
    password='1212',
    database='airline_db',
    charset='utf8mb4'
)

# Query data from the flights_delay table
query = """
SELECT
    flight_id,
    departure_delay_mins,
    delay_minutes AS arrival_delay_mins,
    weather_condition,
    day_of_week,
    aircraft_type,
    airline_code
FROM
    flights_delay
WHERE
    departure_delay_mins IS NOT NULL
    AND delay_minutes IS NOT NULL
    AND weather_condition IS NOT NULL;
"""

# Execute the query and load data into a DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()
# print("Database connection closed.")
# print(f"Data loaded successfully. Total records: {len(df)}")
# print("\nData types:")
# print(df.dtypes)
# print("\nFirst 5 rows:")
# print(df.head())


In [ ]:
df.head()

## Step 2: Data Cleaning and Preparation
We clean the data by removing any missing values and filtering out extreme outliers, which could negatively impact the model's predictive power.


In [ ]:
# Check for missing values before cleaning
print("Missing values per column:")
print(df.isnull().sum())

# Only drop rows with missing values in critical columns
df_cleaned = df.dropna(subset=['departure_delay_mins', 'arrival_delay_mins', 'weather_condition'])

print(f"\nOriginal data: {len(df)} records")
print(f"Cleaned data: {len(df_cleaned)} records")
print(f"Removed records: {len(df) - len(df_cleaned)}")
print("Shape of the final dataset:", df_cleaned.shape)

# Use cleaned dataset for further analysis
df_cleaned

## Step 3: Build the Multiple Regression Model
Build the multiple regression model using `smf.ols`.

dv: arrival_delay_mins ivs: departure_delay_mins,weather_condition

Inside the formula, Use the syntax `C(weather_condition, Treatment(reference='Clear'))` to explicitly set 'Clear' as the baseline category for weather.


In [ ]:
# Q1 - Build the model using 'Clear' as the reference category (note the capital C)

formula ="arrival_delay_mins ~departure_delay_mins + C(weather_condition, Treatment(reference='Clear'))"


In [ ]:
model = smf.ols(formula=formula, data=df_cleaned)
results=model.fit()

## Step 4: Review and Interpret the Results
The model summary will show the coefficient for `departure_delay_mins` as well as the additional impact of other weather conditions compared to 'Clear' weather.


In [ ]:
# Q2 - Print the regression results summary


results.summary()


## Step 5: Visualize the Results
We analyze the model with two visualizations:
1.  A scatter plot comparing the actual and predicted values to check the overall predictive power of the model.
2.  A bar chart comparing the average arrival delay for each weather condition.


In [ ]:
# 1. Visualize the overall model fit (Actual vs. Predicted)
df_cleaned['predicted_delay'] = results.fittedvalues

plt.figure(figsize=(10, 6))
sns.scatterplot(x='predicted_delay', y='arrival_delay_mins', data=df_cleaned)
plt.plot([df_cleaned['arrival_delay_mins'].min(), df_cleaned['arrival_delay_mins'].max()],
         [df_cleaned['arrival_delay_mins'].min(), df_cleaned['arrival_delay_mins'].max()],'r--', lw=2)


In [ ]:
df_cleaned.groupby('weather_condition')['arrival_delay_mins'].mean().sort_values().plot.bar()

In [ ]:
# 2. Visualize the effect of weather condition on arrival delay
plt.figure(figsize=(10, 3))
weather_means = df_cleaned.groupby('weather_condition')['arrival_delay_mins'].mean().sort_values()
sns.barplot(x=weather_means.index, y=weather_means.values)
plt.title('Average Arrival Delay by Weather Condition')
plt.xlabel('Weather Condition')
plt.ylabel('Average Arrival Delay (minutes)')


In [ ]:
# Print summary statistics by weather condition
df_cleaned.groupby('weather_condition')['arrival_delay_mins'].describe()

# Task
Upload the current notebook to GitHub.

## Set up github in colab

### Subtask:
Authenticate Colab with your GitHub account and clone your repository.


## Save the notebook to github

### Subtask:
Save the current notebook to your cloned repository.


## Summary:

### Data Analysis Key Findings

* The attempt to set up GitHub in Colab failed because it requires manual interaction with the Colab user interface for authentication and cloning.
* Saving the notebook to GitHub also failed as it necessitates using the "File" menu in Colab, which is not accessible to the AI.

### Insights or Next Steps

* The user needs to perform the GitHub authentication and repository cloning steps manually within the Colab environment.
* To save the notebook to GitHub, the user must use the "File" menu within Colab and select the "Save a copy in GitHub" option.
